In [25]:
#%run ./lib.py

In [26]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, DecimalType, TimestampType, DataType, IntegerType
from pyspark.sql.functions import col, lit, trim, substring, concat, udf, upper, initcap
from datetime import datetime
import os

In [27]:
conf = SparkConf() \
    .setAppName("projeto_pbi") \
    .setSparkHome('./spark/home')

LAKE_HOME = os.getenv("LAKE_HOME", "/spark/home")

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

In [28]:
TABLE_NAME = 'dim_product'

def create_sk(spark, df, key_column_name, table_name):
    sk = {}
    sk = df.select(col(key_column_name).alias("key")).rdd.zipWithIndex()
    new_sk = sk.map(lambda row: list(row[0]) + [row[1] + 1])
    new_sk_map = new_sk.collectAsMap()

    sk_schema = \
        StructType(
            [StructField('key', StringType(), True),
            StructField('SK', LongType(), True)]
        )

    sk_frame = spark.createDataFrame(new_sk, sk_schema)
    sk_frame.write.mode('overwrite').csv('{}/dataset/e-commerce/02_surrogate_key/sk_{}.csv'.format(LAKE_HOME, table_name), header=True)

    return new_sk_map

def locate_sk(mapping: dict):
    return udf(lambda x: mapping.get(x), IntegerType())

In [29]:
df = spark.read.csv('{}/dataset/e-commerce/01_extract/products_dataset.csv'.format(LAKE_HOME), header=True)

df_product = \
    df.select(
        col('product_id').cast(StringType()).alias('ID_PRODUCT'),
        col('product_category_name').cast(StringType()).alias('DES_CATEGORY'),
        col('product_name_lenght').cast(StringType()).alias('DES_PRODUCT_SHORT'),
        col('product_description_lenght').cast(StringType()).alias('DES_PRODUCT_LONG'),
        col('product_weight_g').cast(DecimalType()).alias('VOL_WEIGHT_G'),
        col('product_length_cm').cast(DecimalType()).alias('DIM_LENGTH_CM'),
        col('product_height_cm').cast(DecimalType()).alias('DIM_HEIGHT_CM'),
        col('product_width_cm').cast(DecimalType()).alias('DIM_WIDTH_CM')
    )

sk = create_sk(spark=spark, df=df_product, key_column_name='ID_PRODUCT', table_name=TABLE_NAME)

In [30]:
dim_product = \
    df_product \
        .withColumn('SK_PRODUCT', locate_sk(sk)(col('ID_PRODUCT')))

In [31]:
dim_product = \
    dim_product.select(
        col('SK_PRODUCT'),
        col('ID_PRODUCT'),
        col('DES_CATEGORY'),
        col('DES_PRODUCT_SHORT'),
        col('DES_PRODUCT_LONG'),
        col('VOL_WEIGHT_G'),
        col('DIM_LENGTH_CM'),
        col('DIM_HEIGHT_CM'),
        col('DIM_WIDTH_CM'),   
    )

dim_product.write.mode('overwrite').csv('{}/dataset/e-commerce/03_dim/{}.csv'.format(LAKE_HOME, TABLE_NAME), header=True)